In [ ]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv(r"*.csv", dtype = np.float32)

In [ ]:
X = data.iloc[:, 1:11]
Y = data.iloc[:, 11:13]

In [ ]:
train_stats = Y.describe()
train_stats = train_stats.transpose()
train_stats

In [ ]:
X_normal = preprocessing.scale(X, axis=0)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X_normal, Y.values, train_size = 0.8)

In [ ]:
Y = None
X = None

In [ ]:
import tensorflow.keras.backend as K
def r2(y_true, y_pred):
    a = K.square(y_pred - y_true)
    b = K.sum(a)
    c = K.mean(y_true)
    d = K.square(y_true - c)
    e = K.sum(d)
    f = 1 - b / e
    return f

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(128, activation="relu", input_shape=(10,)))
model.add(tf.keras.layers.Dense(128, activation="relu"))
model.add(tf.keras.layers.Dense(2, activation="linear"))
model.summary()

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01, beta_1=0.9, beta_2=0.99)
model.compile(loss='mse', optimizer=optimizer, metrics=['RootMeanSquaredError', r2])
history = model.fit(X_train, Y_train, batch_size=512, epochs=100, 
                    validation_split=0.2, validation_freq=1)

In [ ]:
history=history.history

In [ ]:
y_loss = history['loss']
y_rmse = history['r2']
y_v_loss = history['val_loss']
y_v_rmse = history['val_r2']
x_epoch = [i for i in range(1,len(y_loss)+1)]

In [ ]:
plt.plot(x_epoch, y_loss,color='red',linewidth=1.0,linestyle='-',label='training')
plt.plot(x_epoch, y_v_loss,color='blue',linewidth=1.0,linestyle='-',label='val')
plt.title("Tendency of Loss")
plt.xlabel("Number of Training")
plt.ylabel("Loss")
plt.legend()

In [ ]:
plt.plot(x_epoch,y_rmse,color='red',linewidth=1.0,linestyle='-',label='training')
plt.plot(x_epoch,y_v_rmse,color='blue',linewidth=1.0,linestyle='-',label='val')
plt.title("Tendency of R2")
plt.xlabel("Number of Training")
plt.ylabel("R2")
plt.legend()

In [ ]:
pred_X = model.predict(X_test)

In [ ]:
r2 = r2_score(y_true=Y_test, y_pred=pred_X)
mse = mean_squared_error(y_true=Y_test, y_pred=pred_X)
mape = mean_absolute_percentage_error(y_true=Y_test, y_pred=pred_X)

print('r2 = ', r2, '\nmse = ', mse, '\nmape = ', mape)

In [ ]:
r2_c = r2_score(y_true=Y_test[:, 0], y_pred=pred_X[:, 0])
mse_c = mean_squared_error(y_true=Y_test[:, 0], y_pred=pred_X[:, 0])
mape_c = mean_absolute_percentage_error(y_true=Y_test[:, 0], y_pred=pred_X[:, 0])

r2_s = r2_score(y_true=Y_test[:, 1], y_pred=pred_X[:, 1])
mse_s = mean_squared_error(y_true=Y_test[:, 1], y_pred=pred_X[:, 1])
mape_s = mean_absolute_percentage_error(y_true=Y_test[:, 1], y_pred=pred_X[:, 1])

r2_a = r2_score(y_true=Y_test[:, 0] + Y_test[:, 1], y_pred=pred_X[:, 0] + pred_X[:, 1])
mse_a = mean_squared_error(y_true=Y_test[:, 0] + Y_test[:, 1], y_pred=pred_X[:, 0] + pred_X[:, 1])
mape_a = mean_absolute_percentage_error(y_true=Y_test[:, 0] + Y_test[:, 1], y_pred=pred_X[:, 0] + pred_X[:, 1])

print('LEc\nr2 = ', r2_c, '\nmse = ', mse_c, '\nmape = ', mape_c)
print('LEs\nr2 = ', r2_s, '\nmse = ', mse_s, '\nmape = ', mape_s)
print('LE\nr2 = ', r2_a, '\nmse = ', mse_a, '\nmape = ', mape_a)


In [ ]:
model.save(r"*.h5")